torchtext는 자연어 처리 분야에서 사용하는 데이터로더이다.파일 가져오기, 토큰화, 단어 집합 생성, 인코딩, 단어 벡터 생성 등의 작업을 지원하기 때문에 자연어 처리에서 많이 사용되고 있다.

In [3]:
import torch
import torchtext # 0.9
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

## 데이터 전처리

In [5]:
start = time.time()
TEXT = torchtext.legacy.data.Field(lower=True, fix_length=200, batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False)

* torchtext.legacy.data에서 제공하는 Field는 데이터 전처리를 위해 사용되며 여기에서 사용되는 파라미터는 다음과 같다.
    1. lower: 대문자를 모두 소문자로 변경한다. 기본값은 false
    2. fix_length: 고정된 길이의 데이터를 얻을 수 있다. 여기에서는 데이터의 길이를 200으로 고정했으며 200보다 짧다면 패딩을 시켜준다.
    3. batch_first: 신경망에 입력되는 텐서의 첫 번째 차원 값이 배치 크기가 되도록 한다.

## 데이터셋 준비

In [6]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:15<00:00, 5.54MB/s]


In [7]:
print(vars(train_data.examples[0])) # 데이터셋의 내용을 보고자 할 때는 examples를 사용

{'text': ['for', 'the', 'life', 'of', 'me,', 'why', 'did', 'this', 'film', 'receive', 'an', 'r', 'rating?!', 'while', 'it', 'is', 'about', 'flesh-eating', 'zombies,', 'believe', 'it', 'or', 'not,', "it's", 'actually', 'a', 'pretty', 'good', 'family-friendly', 'film--at', 'least', 'if', 'your', 'kids', 'are', 'age', '10', 'and', 'older.', 'unlike', 'the', 'traditional', 'zombie', 'films,', 'this', 'one', 'has', 'an', 'excellent', 'sense', 'of', 'humor', 'as', 'well', 'as', 'a', 'traditional', 'values--albeit', 'a', 'bit', 'twisted!', 'the', 'language', "isn't", 'a', 'serious', 'problem,', 'there', 'is', 'no', 'nudity', 'and', 'the', 'film', 'style', 'is', 'definitely', 'geared', 'towards', 'kids', '(much', 'like', 'the', 'old', 'tv', 'show', '"eerie,', 'indiana")--yet', 'some', 'knucklehead', 'slapped', 'an', 'r', 'rating', 'one', 'it!', 'believe', 'me,', 'most', 'kids', 'have', 'seen', 'worse', 'violence', 'than', 'this', 'and', 'it', 'just', 'seems', 'silly', 'to', 'make', 'audiences'

## 데이터셋 전처리 적용

In [9]:
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']] # 소문자로 변경
    text = [x.replace('<br', '') for x in text] # '<br'을 ''으로 변경
    text = [''.join(c for c in s if c not in string.punctuation) for s in text] # 구두점 제거
    text = [s for s in text if s] # 공백 제거
    vars(example)['text'] = text

## 훈련과 검증 데이터셋 분리

In [10]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [11]:
print('Number of training examples', len(train_data))
print('Number of validation examples', len(valid_data))
print('Number of testing examples', len(test_data))

Number of training examples 20000
Number of validation examples 5000
Number of testing examples 25000


## 단어 집합 만들기

In [12]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)